In [1]:
import re, os

Create corpus (.txt file) for each language

In [2]:
def clean_sentences(content):
    # Remove opening html tag + article title
    trimmed_open = re.sub(r'<doc\b[^>]*>\s*(?:.*\n)?', '', content)
    # Remove closing html tag
    trimmed_close = re.sub(r'<\/doc\b[^>]*>', '', trimmed_open)
    # Split at periods while trying to overlook abbreviations
    return re.split(r'(?<!\w\.\w)(?<!\w\.)\.(?!\w)', trimmed_close)

In [3]:
# regex to exclude tree diagrams in wiki
# (?<=<doc\b[^>]*>\s*(?:.*\n)?\n)(.*?)(?=\n\n<\/doc>)

In [4]:
# Directory with extracted files
maindir = '/Users/k/Docs/School/Tuebingen/Thesis/Corpuses/HtmlWiki'
wikidirs = [sub for sub in os.listdir(maindir) if sub != '.DS_Store']
# Dict of languages + their corresponding alphabets
alphabets = {'lfn': re.compile(r'[^a-z]'), 'ia': re.compile(r'[^a-z]'), 'eo': 'abcĉdefgĝhĥijĵklmnoprsŝtuŭvz', 'de': 'abcdefghijklmnopqrstuvwxyzäöüß', 'fr': 'abcdefghijklmnopqrstuvwxyzéàèùëüïâêîôûçæœ', 'en': re.compile(r'[^a-z]')}

# Estimated smallest wc (from lfn corpus)
min_wc = 1265000

for wiki in wikidirs:
    wiki_fullpath = os.path.join(maindir, wiki)
    lang = os.path.splitext(os.path.basename(wiki))[0].split('_')[0]
    current_wc = 0
    if lang in alphabets:
        alpha = alphabets[lang]
    with open(f'/Users/k/Docs/School/Tuebingen/Thesis/iscl-thesis/current_corpora/{lang}_wiki_html.txt', 'w', encoding='utf-8') as out:
        for root, dirs, files in os.walk(wiki_fullpath):
            for file in files:
                file_path = os.path.join(root, file)
                if '/wiki' in file_path:
                    with open(file_path) as f: 
                        content = f.read()
                        f.close()
                    sentences = clean_sentences(content)
                    # Check if alpha in dict is regex
                    if isinstance(alpha, re.Pattern):
                        for sentence in sentences:
                            # Remove Wiki article subheaders
                            if len(sentence.split()) > 1:
                                # Remove all chars not language's alphabet
                                s = ' '.join([re.sub(alpha, '', word.strip().lower()) for word in sentence.split()])
                                # Remove leftover whitespaces
                                s = re.sub(r'\s+', ' ', s).strip()
                                if (len(s.split()) + current_wc) < min_wc:
                                    current_wc += len(s.split())
                                    out.write(s + '.' + '\n')
                                else:
                                    break
                            else:
                                continue
                    else:
                        for sentence in sentences:
                            # Remove Wiki article subheaders
                            if len(sentence.split()) > 1:
                                # Remove all chars not language's alphabet
                                s = ' '.join([word.lower() for word in sentence.split() if all(letter in alpha for letter in word.lower())])
                                # Remove leftover whitespaces
                                s = re.sub(r'\s+', ' ', s).strip()
                                if (len(s.split()) + current_wc) < min_wc:
                                    current_wc += len(s.split())
                                    out.write(s + '.' + '\n')
                                else:
                                    break
                            else:
                                continue
    out.close()